In [1]:
import pandas as pd

train_df = pd.read_csv('./datas/ratings_train.txt', sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [2]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [3]:
import re
train_df = train_df.fillna(' ')
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r"\d+", " ",x))

In [4]:
test_df = pd.read_csv('./datas/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+", " ",x))

In [5]:
from konlpy.tag import Okt
t = Okt()
def t_tokenizer(text):
    tokens_ko = t.morphs(text)
    return tokens_ko

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

c:\users\luvu1\anaconda3\envs\nltk\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [12]:
from lightgbm import LGBMClassifier
import time

start_time = time.time()
lgbm_clf = LGBMClassifier(n_estimators=400)
lgbm_clf.fit(tfidf_matrix_train, train_df['label'])
print('Fit time : ', time.time() - start_time)

Fit time :  102.94339513778687


In [15]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])
preds = lgbm_clf.predict(tfidf_matrix_test)
accuracy_score(test_df['label'], preds)

0.82958

In [16]:
test_df['document'][100]

'걸작은 몇안되고 졸작들만 넘쳐난다.'

In [17]:
lgbm_clf.predict(tfidf_vect.transform([test_df['document'][100]]))

array([0], dtype=int64)

In [18]:
lgbm_clf.predict(tfidf_vect.transform(['신기하다']))

array([0], dtype=int64)

In [19]:
lgbm_clf.predict(tfidf_vect.transform(['짜증나']))

array([0], dtype=int64)

In [20]:
lgbm_clf.predict(tfidf_vect.transform(['좋다']))

array([1], dtype=int64)